In [ ]:
import numpy as np

## Mendefiniskan Environment

Environment terdiri dari **States**, **Actions**, dan **Rewards**. States dan Actions adalah input untuk agent Q-learning, sedangkan tindakan yang mungkin terjadi adalah output agent.

#### States
States di dalam lingkungan adalah semua kemungkinan lokasi di dalam gudang. Beberapa lokasi digunakan untuk menyimpan barang (**kotak hitam**), sementara lokasi yang lain adalah lorong yang digunakan robot untuk berjalan (**kotak putih**). Kotak hijau menunjukkan area pengemasan dan pengiriman barang (**goals**).

Kotak hitam dan kotak hijau adalah **terminal states**

![warehouse map](https://www.danielsoper.com/teaching/img/08-warehouse-map.png)

Tujuan agent adalah mempelajari jalur terpendek antara area pengemasan barang (kotak hijau) dan semua lokasi lain di gudang tempat robot diizinkan untuk bepergian (kotak putih).

Seperti yang ditunjukkan gambar di atas, ada 121 kemungkinan states (lokasi) di dalam gudang. States disusun dalam kisi yang berisi 11 baris dan 11 kolom. Setiap lokasi dapat diidentifikasi oleh indeks baris dan kolom.

In [ ]:
#mendefinisikan bentuk environment yang memiliki 11 rows, dan 11 columns, sehingga memiliki 121 kemungkinan states (lokasi)
env_rows = 11
env_columns = 11

"""
Create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a) 
The array contains 11 rows and 11 columns (to match the shape of the environment), as well as a third "action" dimension.
The "action" dimension consists of 4 layers that will allow us to keep track of the Q-values for each possible action in each state (see next cell for a description of possible actions). 
The value of each (state, action) pair is initialized to 0.
"""
q_values = np.zeros((env_rows, env_columns, 4))

### Actions ###
Actions yang tersedia untuk agent adalah bergerak ke empat arah:
* Atas
* Kanan
* Bawah
* Kiri

Agent juga harus belajar untuk menghindari lokasi penyimpanan barang (kotak hitam)

In [ ]:
#mendefinisikan action
#up = 1, right = 2, down = 3, left = 4
actions = ['up', 'right', 'down', 'left']

### Rewards ###

Komponen terakhir dari environment adalah mendefinisikan **Rewards**.

Untuk membantu agent belajar, setiap **States** (lokasi) diberi nilai **Reward**

Agent bisa saja memulai dari kotak putih manapun, namun tujuannya selalu sama yaitu **memaksimalkan total rewards**

**Negative rewards** (punishment) digunakan pada semua states kecual goal
* Hal ini akan mendorong AI untuk mengidentifikasi jalur terpendek ke tujuannya dengan meminimalkan punishment.

![warehouse map](https://www.danielsoper.com/teaching/img/08-warehouse-map-rewards.png)

Untuk memaksimalkan cumulative rewards (dengan meminimalkan cumulative punishment) Agent perlu menemukan jalur terpendek antara area pengemasan barang (**kotak hijau**) dan semua lokasi lain di gudang tempat robot diizinkan untuk bepergian (kotak putih). Agent juga perlu belajar untuk menghindari menabrak lokasi penyimpanan (**kotak hitam**)



In [ ]:
#membuat array 2 dimensi untuk menapung reward
#array berisi 11 baris dan 11 kolom (sesuai dengan environment), dan diberi nilai -100
rewards = np.full((env_rows, env_columns), -100.)
rewards [0, 5] = 100. #memberi nilai reward 100 pada goal

#menentukan lorong yang akan diberi reward -1
aisles = {} #store locations in a dictionary
aisles[1] = [i for i in range (1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 9)]
aisles[3].append(9) 
aisles[4] = [3, 7]
aisles[5] = [i for i in range (11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]

#nested for untuk memberi nilai reward untuk tiap lorong
for row_index in range(1, 10):
    for column_index in aisles[row_index]:
        rewards[row_index, column_index] = -1

#menampilkan reward
for row in rewards:
    print(row)

### Melatih model ###

Langkah selanjutnya adalah agar agent mempelajari lingkungannya dengan menerapkan model Q-learning. proses pembelajaran akan mengikuti langkah-langkah berikut:

1. Pilih non-terminal state (kotak putih) secara acak untuk agent memulai episode
2. pilih actions (bergerak ke atas, kanan, bawah, kiri) untuk state saat ini. Tindakan akan dipilih menggunakan epsilon greedy algorithm. Algoritma ini akan memilih tindakan yang paling menguntungkan untuk agent, tapi terkadang akan memilih opsi kurang menguntungkan untuk mendorong agent mengeksplor environtment.
3. Melakukan tindakan yang dipilih, transisi ke state selanjutnya (pindah ke lokasi berikutnya).
4. Mendapatkan reward setelah pindah ke state (lokasi) baru, dan menghitung temporal difference
5. Memperbarui Q-value untuk state dan action sebelumnya.
6. Jika state baru (saat ini) adalah terminal-state (kotak hitam/hijau) maka masuk ke step 1, jika bukan maka masuk ke step 2.

Seluruh proses akan diulang sebanyak 1000 episode. Ini akan memberi agent kesempatan untuk mempelajari jalur terpendek antara area pengemasan barang (**kotak hijau**) dan semua lokasi lain di gudang tempat robot diizinkan bepergian (**kotak putih**), sekaligus menghindari lokasi penyimpanan barang (**kotak hitam**)


In [ ]:

#non terminal state = kotak putih/yang nilainya -1
#terminal state = kotak hitam/yang nilaainya -100 & goal yang nilainya 100

#fungsi untuk menentukan menentukan apakah lokasi yang ditentukan adalah terminal state
def is_terminal_state(current_row_index, current_column_index):
    if rewards[current_row_index, current_column_index] == -1.: #jika reward = -1 maka bukan terminal state
        return False
    else:
        return True

#fungsi menentukan lokasi awal secara acak (yang non terminal state/kotak putih)
def get_starting_location():

    #mendapatkan baris acak
    current_row_index = np.random.randint(env_rows)
    current_column_index = np.random.randint(env_columns)

    #terus memilih baris dan kolom secara acak sampai ketemu non terminal state/kotak putih
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(env_rows)
        current_column_index = np.random.randint(env_columns)
    return current_row_index, current_column_index

#mendefinisikan epsilon greedy algorithm
def get_next_action(current_row_index, current_column_index, epsilon):

    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else:#10% nya memilih action secara random (untuk eksplorasi)
        return np.random.randint(4)

#mendefinisikan lokasi berikutnya setelah melakukan action yang dipilih
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < env_columns -1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < env_rows -1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index

#fungsi untuk menentukan jalur terpendek yang dapat dilalui dari lokasi manapun yang diizinkan (lokasi yang kotak putih)
def get_shortest_path(start_row_index, start_column_index):

    if is_terminal_state(start_row_index, start_column_index):
        return []
    else:
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = []
        shortest_path.append([current_row_index, current_column_index])

        while not is_terminal_state(current_row_index, current_column_index):
            action_index = get_next_action(current_row_index, current_column_index, 1.)

            current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
            shortest_path.append([current_row_index, current_column_index])
            
        return shortest_path

#### Training agent dengan menggunakan Q-learning ####

In [ ]:
#mendefiniskan training parameter
epsilon = 0.9 
discount_factor = 0.9
learning_rate = 0.9

#menjalankan sebanyak 1000 episode
for episode in range (1000):

    #mendapatkan lokasi awal untuk episode ini
    row_index, column_index = get_starting_location()

    #terus melakukan action sampai mencapai terminal state (kotak hitam/hijau)
    while not is_terminal_state(row_index, column_index):
        action_index = get_next_action(row_index, column_index, epsilon)

        #menentukan aksi dan memilih keadaan selanjutnya (berpindah ke lokasi selanjutnya)
        old_row_index, old_column_index = row_index, column_index
        row_index, column_index = get_next_location(row_index, column_index, action_index)
        
        #mendapatkan hadiah dan menghitung temporal difference
        reward = rewards[row_index, column_index]
        old_q_values = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_values
        
        #update nilai q
        new_q_value = old_q_values + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete')

### Mendapatkan Jalur Terpendek ###

![warehouse map](https://www.danielsoper.com/teaching/img/08-warehouse-map.png)

In [ ]:
print(get_shortest_path(9, 3)) #dimulai pada baris ke 9, kolom 3
print(get_shortest_path(5, 0)) #dimulai dari baris ke 5, kolom 0
print(get_shortest_path(9, 5)) #dimulai dari baris ke 9, kolom 5


In [ ]:
#dibalik
path = get_shortest_path(5, 2)
path.reverse()
print("")
print(path)